# Kaggle Competition
## Any Bossom at Home?

We have been asked by CERN physicists to help them uncover important particle traces. We are provided data from the Large Hadron Collider B, where protons collide at 99.9999% of the speed of the light. There are 40M of collisions per second, and each collision produces from hundreds to thousands of particles. From those only a very small set are of interest. We are looking for a process of B0 into K*0 gamma decomposition. Only 10 of these processes are produced every our. Can we uncover them from the sea of processes?

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model, ensemble, tree, svm, neighbors, neural_network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, MinMaxScaler, QuantileTransformer, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, roc_auc_score

%matplotlib inline

## Read data and Split into train and Test

In [14]:
df = pd.read_csv('../data/train.csv', index_col=0)
df.columns = df.columns.str.strip()
print(df.columns)

# Low separation
# del df['piminus_ETA']
# del df['Kplus_ETA']

# WTF is this?
del df['BUTTER']

X = df[df.columns[:-1]]
y = df[df.columns[-1]]

y.value_counts()

Index(['B_OWNPV_CHI2', 'B_IPCHI2_OWNPV', 'B_FDCHI2_OWNPV', 'B_DIRA_OWNPV',
       'B_PT', 'Kst_892_0_IP_OWNPV', 'Kst_892_0_cosThetaH', 'Kplus_IP_OWNPV',
       'Kplus_P', 'piminus_IP_OWNPV', 'piminus_P', 'gamma_PT', 'piminus_ETA',
       'Kplus_ETA', 'BUTTER', 'signal'],
      dtype='object')


0.0    141632
1.0     71030
Name: signal, dtype: int64

## Explore Model selection

In [20]:
models = [
    linear_model.LinearRegression(),
    linear_model.RidgeClassifier(),
    tree.DecisionTreeClassifier(),
    ensemble.RandomForestClassifier(min_samples_leaf=5, class_weight={0:2, 1:3}),
    ensemble.AdaBoostClassifier(),
    ensemble.GradientBoostingClassifier(),
    neighbors.KNeighborsClassifier()
]


scalers = [
    None,
    Normalizer(),
    MinMaxScaler(),
    StandardScaler(),
    QuantileTransformer(),
    RobustScaler(),
]

def score_model(X, y, model, scaler):   
    print('%s;%s;'%(model, scaler), end='')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

    if scaler is not None:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)

    # Binarize maybe
    y_hat[y_hat >= 0.5] = 1
    y_hat[y_hat < 0.5] = 0

    accuracy = accuracy_score(y_test, y_hat)
    if 'predict_proba' in dir(model):
        roc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    else:
        roc_score = roc_auc_score(y_test, model.predict(X_test))
        
    print("%f;%f"%(accuracy,roc_score))

def cross_validate(X, y, model, scaler):
    scores = []
    print('%s;%s;'%(model, scaler), end='')
    
    for i in range(4):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=i)

        if scaler is not None:
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        model.fit(X_train, y_train)
        y_hat = model.predict(X_test)

        # Binarize maybe
        y_hat[y_hat >= 0.5] = 1
        y_hat[y_hat < 0.5] = 0
        
        scores += [model.score(X_test, y_test)]
    final_score = sum(scores) / len(scores)
    print(final_score)
    
for model in models:
    for scaler in scalers:
        score_model(X, y, model, scaler)

LinearRegression();None;0.734608;0.787417
LinearRegression();Normalizer();0.743009;0.799472
LinearRegression();MinMaxScaler();0.734608;0.787417
LinearRegression();StandardScaler();0.734608;0.787417
LinearRegression();QuantileTransformer();0.756834;0.817446
LinearRegression();RobustScaler();0.734608;0.787417
RidgeClassifier();None;0.734545;0.658992
RidgeClassifier();Normalizer();0.684013;0.561407
RidgeClassifier();MinMaxScaler();0.734828;0.659507
RidgeClassifier();StandardScaler();0.734608;0.659435
RidgeClassifier();QuantileTransformer();0.756740;0.695968
RidgeClassifier();RobustScaler();0.734608;0.659435
DecisionTreeClassifier();None;0.690784;0.655502
DecisionTreeClassifier();Normalizer();0.675643;0.639073
DecisionTreeClassifier();MinMaxScaler();0.692226;0.657331
DecisionTreeClassifier();StandardScaler();0.689687;0.653909
DecisionTreeClassifier();QuantileTransformer();0.688119;0.652499
DecisionTreeClassifier();RobustScaler();0.687774;0.651844
RandomForestClassifier(class_weight={0: 2, 

### Scoring the best model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

model = ensemble.RandomForestClassifier()
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model.fit(X_train, y_train)
y_hat = model.predict(X_test)

print('Accuracy', accuracy_score(y_test, y_hat))
print('AUC Score', roc_auc_score(y_test, y_hat))

0.722523091131483

In [7]:
print('Accuracy', accuracy_score(y_test, y_hat))
print('AUC Score', roc_auc_score(y_test, y_hat))

Accuracy 0.7719749216300941
AUC Score 0.722523091131483


## Explore Scaler

In [118]:
scaler = Normalizer()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)
X_train_s = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns)
X_test_s = pd.DataFrame(data=scaler.transform(X_test), columns=X_train.columns)

X_train_s.describe()

,B_OWNPV_CHI2,B_IPCHI2_OWNPV,B_FDCHI2_OWNPV,B_DIRA_OWNPV,B_PT,Kst_892_0_IP_OWNPV,Kst_892_0_cosThetaH,Kplus_IP_OWNPV,Kplus_P,piminus_IP_OWNPV,piminus_P,gamma_PT,piminus_ETA,Kplus_ETA
count,180762.000000,1.807620e+05,180762.000000,1.807620e+05,180762.000000,1.807620e+05,180762.000000,1.807620e+05,180762.000000,1.807620e+05,180762.000000,180762.000000,180762.000000,180762.000000
mean,0.000938,8.451895e-05,0.077699,3.013230e-05,0.257533,1.712249e-05,0.000007,1.738378e-05,0.681678,1.765928e-05,0.558923,0.168696,0.000091,0.000091
std,0.000700,8.313704e-05,0.139527,1.453395e-05,0.122812,1.494780e-05,0.000017,1.567776e-05,0.186436,1.597446e-05,0.207707,0.098239,0.000037,0.000036
min,0.000006,3.903538e-10,0.000406,5.073240e-07,0.003455,4.639057e-07,-0.000062,2.664117e-07,0.003170,4.143417e-07,0.002804,0.002355,0.000001,0.000001
25%,0.000433,2.295012e-05,0.010175,1.867240e-05,0.160016,7.213157e-06,-0.000004,7.111546e-06,0.530055,7.219154e-06,0.389654,0.093428,0.000062,0.000063
50%,0.000756,5.845566e-05,0.024951,2.751129e-05,0.238416,1.225738e-05,0.000006,1.226963e-05,0.698846,1.249709e-05,0.569460,0.146518,0.000086,0.000087
75%,0.001254,1.192279e-04,0.073853,3.910674e-05,0.336636,2.157760e-05,0.000017,2.187809e-05,0.844323,2.219600e-05,0.737950,0.221546,0.000115,0.000115
max,0.007259,7.021951e-04,0.999980,1.012164e-04,0.715704,1.945528e-04,0.000092,2.834155e-04,0.992423,2.834155e-04,0.943568,0.662169,0.000300,0.000273


In [70]:
err = np.abs(y_hat - y_test.values)

acc = (y_test.count() - err.sum()) / y_test.count()
print("Accuracy: %.3f%%" % (acc * 100))

Accuracy: 76.555%


In [74]:
print(scaler)
print(ln_model)
ln_model.score(X_test, y_test)

Normalizer()
RandomForestClassifier()


0.7655485893416928

## Pair Plots

We can plot the data using Seaborn's "Pair Plotting" to get a very nice visualization of correlation graphs for each variable against each other, as well as density graphs for cases where the prediction variable is one value or the other (the "hue" parameter)

In [ ]:
df = pd.read_csv('data/train.csv', index_col=0)
df.columns = df.columns.str.strip()
del df['BUTTER']

sns.pairplot(df[::1000], hue='signal')
plt.show()

**We observe a couple of interesting facts about our features:**

1. There is an extremely suspicious variable called `Butter` that separates almost perfectly the two groups, better than any other feature. If we take a look at it's description we get: `BUTTER: Corresponds to the relative consumption of butter cream in Switzerland.` Nonsense. It is probably a trap and the data looks generated by a standard normal distribution with posterior knowledge of the "signal" target. **We should discard it**.


2. Some of the features **completely overlap their density graphs on both signals 1 and 0**. These features can at best be ignored by our model and at worst cause numerical instability or bias errors. **We should try to train the models with these features on or off** to see which case performs better.


3. Some features show promise of having a good correlation with signal. We can observe this when the Pair plots have **distinct colored clusters** and their **density graphs don't overlap** in some areas.


4. Could we do some post-processing of the features showing an exponential distribution to bring into a log-normalized space? Would that help prediction accuracy?